In [ ]:
import osmnx as ox
import networkx as nx
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. TẢI DỮ LIỆU TỪ OPENSTREETMAP
city_name = "Tokyo, Japan"  # Bạn có thể đổi thành phố khác
G = ox.graph_from_place(city_name, network_type="drive")

# 2. CHUYỂN ĐỔI ĐỒ THỊ SANG DỮ LIỆU PYTORCH GEOMETRIC
node_mapping = {node: i for i, node in enumerate(G.nodes)}
edge_index = []
node_features = []

# Duyệt từng node để lấy đặc trưng (ví dụ: tọa độ)
for node in G.nodes(data=True):
    lat, lon = node[1]["y"], node[1]["x"]  # Lấy tọa độ
    node_features.append([lat, lon])  # Có thể mở rộng feature nếu cần

# Chuẩn hóa node features
scaler = StandardScaler()
node_features = scaler.fit_transform(node_features)
node_features = torch.tensor(node_features, dtype=torch.float)

# Tạo danh sách cạnh (edges)
for u, v, _ in G.edges(data=True):
    edge_index.append([node_mapping[u], node_mapping[v]])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Tạo graph data cho PyTorch Geometric
data = Data(x=node_features, edge_index=edge_index)

# 3. ĐỊNH NGHĨA MÔ HÌNH GNN
class CityLayoutGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(CityLayoutGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Khởi tạo mô hình
in_channels = node_features.shape[1]
model = CityLayoutGNN(in_channels, 64, in_channels)

# 4. HUẤN LUYỆN MÔ HÌNH
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.mse_loss(out, data.x)  # Reconstruction loss
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# 5. LƯU MÔ HÌNH VÀ CHUYỂN SANG ONNX
dummy_input = (data.x, data.edge_index)
torch.onnx.export(model, dummy_input, "city_layout.onnx")
print("Mô hình đã lưu thành công dưới dạng ONNX!")
